In [1]:
#load necessary packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from calendar import IllegalMonthError,IllegalWeekdayError
from dateutil.parser import parse
from unidecode import unidecode

def remove_excessive_ws(x):
    if isinstance(x,str):
        return ' '.join(x.split())
    else:
        return x

def myunidecode(x):
    if isinstance(x,str):
        return unidecode(x)
    else:
        return x
    
def remove_short(x):
    if isinstance(x,str):
        tmp = x.split()
        return ' '.join([_x for _x in tmp if len(_x)>2])
    else:
        return x

In [2]:
# load data of murders of social leaders 
lastrowdropped = False
# load data of murders of social leaders
data_social = pd.read_csv('Data/J_NHomicidiosLideresSocialesColombia2012-2017.csv', encoding='latin-1')

# load homicides
data_2012 = pd.read_csv('Data/Delito_Homicidios_2012.csv',encoding='latin-1')
data_2013 = pd.read_csv('Data/Delito_Homicidios_2013.csv',encoding='latin-1')
data_2014 = pd.read_csv('Data/Delito_Homicidios_2014.csv',encoding='latin-1')
data_2015 = pd.read_csv('Data/Delito_Homicidios_2015.csv',encoding='latin-1')
data_2016 = pd.read_csv('Data/Homicidios_2016.csv',encoding='utf8')
data_2017 = pd.read_csv('Data/Homicidios_2017.csv',encoding='utf8')


#remove rows that contains mostly nan
print( 'This deletion takes care of rows meant as separators that just contain the year')
print ('number of rows before NA deletion: {}'.format(data_social.shape[0]))
data_social.dropna(axis=0,thresh=5,inplace=True)
print( 'number of rows after NA deletion: {}'.format(data_social.shape[0]))

This deletion takes care of rows meant as separators that just contain the year
number of rows before NA deletion: 366
number of rows after NA deletion: 361


In [3]:
if not lastrowdropped:
    #last row contains total count. Drop this.
    lastrowdropped = True
    data_2012 = data_2012.drop(data_2012.index[-1])
    data_2013 = data_2013.drop(data_2013.index[-1])
    data_2014 = data_2014.drop(data_2014.index[-1])
    data_2015 = data_2015.drop(data_2015.index[-1])
    data_2016 = data_2016.drop(data_2016.index[-1])
    data_2017 = data_2017.drop(data_2017.index[-1])
    
# Drop cantidad columns for now
data_2012 = data_2012.drop('2012', axis=1)
data_2013 = data_2013.drop('2013', axis=1)
data_2014 = data_2014.drop('2014', axis=1)
data_2015 = data_2015.drop('2015', axis=1)
data_2016 = data_2016.drop('Cantidad', axis=1)
data_2017 = data_2017.drop('Cantidad', axis=1)


In [4]:
data_2012.columns = [x.strip() for x in data_2012.columns]
data_2012.columns

Index(['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'DIA', 'HORA', 'BARRIO', 'ZONA',
       'CLASE SITIO', 'ARMA EMPLEADA', 'MOVIL AGRESOR', 'MOVIL VICTIMA',
       'EDAD', 'SEXO', 'ESTADO CIVIL', 'PAIS NACE', 'CLASE EMPLEADO',
       'PROFESIONES', 'ESCOLARIDAD', 'CODIGO DANE'],
      dtype='object')

## modify column names to match them

In [5]:
#Modify column names so they match
data_social.columns = [x.strip() for x in data_social.columns]
data_2012.columns = [x.strip() for x in data_2012.columns]
data_2013.columns = [x.strip() for x in data_2013.columns]
data_2014.columns = [x.strip() for x in data_2014.columns]
data_2015.columns = [x.strip() for x in data_2015.columns]
data_2016.columns = [x.strip() for x in data_2016.columns]
data_2017.columns = [x.strip() for x in data_2017.columns]
data_2015.rename(columns={u'ARMA O MEDIO': u'ARMA EMPLEADA', 
                          u'CLASE DE SITIO': u'CLASE SITIO',u'GENERO ':u'SEXO', 
                         u'PROFESION':u'PROFESIONES', u'CODIGO DANE ':u'CODIGO DANE' }, inplace=True)
data_2016.columns = map(myunidecode, data_2016.columns)
data_2016.columns = map(myunidecode, data_2016.columns)
data_2016.columns = map(str.upper, data_2016.columns)
data_2016.rename(columns={u'CLASE DE SITIO': u'CLASE SITIO',u'PROFESION':u'PROFESIONES', 
                          u'CLASE DE EMPLEADO':u'CLASE EMPLEADO',u'PAIS DE NACIMIENTO':u'PAIS NACE' }, inplace=True)
data_2017.columns = map(myunidecode, data_2017.columns)
data_2017.columns = map(str.upper, data_2017.columns)
data_2017.rename(columns={u'CLASE DE SITIO': u'CLASE SITIO',u'PROFESION':u'PROFESIONES', 
                          u'CLASE DE EMPLEADO':u'CLASE EMPLEADO',u'PAIS DE NACIMIENTO':u'PAIS NACE' }, inplace=True)


## concatenate homicide datasets 

In [6]:
data_all = pd.concat((data_2012,data_2013,data_2014,data_2015,data_2016,data_2017))

/Users/ben/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


## remove dates in the social leaders data that cannot be handled and are ambiguous

In [7]:
colnames_social = list(data_social.columns.values)

In [8]:
data_deleted = [data_social[data_social[colnames_social[11]].str.contains("112012")==True],
                          data_social[data_social[colnames_social[11]].str.contains("16710/2011")==True],
                          data_social[data_social[colnames_social[11]].str.contains("REPORTE S/D")==True]]

In [9]:
#remove/change row where date cannot be fixed
data_social = data_social[data_social[colnames_social[11]].str.contains("112012")!=True]
data_social = data_social[data_social[colnames_social[11]].str.contains("16710/2011")!=True]
data_social = data_social[data_social[colnames_social[11]].str.contains("REPORTE S/D")!=True]

data_social[colnames_social[11]].replace(to_replace='15/022015', value='15/02/2015',inplace=True)
data_social[colnames_social[11]].replace(to_replace='26/082016', value='26/08/2016',inplace=True)
data_social[colnames_social[11]].replace(to_replace='07.04.203', value='07/04/2013',inplace=True)
data_social[colnames_social[11]].replace(to_replace='17/04/214', value='17/04/2014',inplace=True)


## remove accents to make NLP processing easier

In [10]:
cols_to_scan = [ u'SECCIONAL', u'NOMBRE VICTIMA', u'APELLIDO',
       u'NOMBRES', u'CALIDAD DE LA VICTIMA', u'TIPO DE ORGANIZACIÓN',
       u'ORGANIZACIÓN A LA QUE PERTENECE', u'NOMBRE DE LA ORGANIZACIÓN',
       u'RANGO DENTRO DE LA ORGANIZACIÓN',
       u'DEPARTAMENTO', u'MUNICIPIO', u'LUGAR ESPECIFICO', u'ARMA UTILIZADA',
       u'INDICIADOS O IMPUTADOS', u'SITUACION JURIDICA IMPUTADOS',
       u'ÓRDENES DE PROTECCIÓN', u'FASE PRCESAL']



for colname in cols_to_scan:
    data_social[colname] = data_social[colname].apply(myunidecode)
    
cols_to_scan = [u'ARMA EMPLEADA', u'BARRIO', u'CLASE EMPLEADO', u'CLASE SITIO',
        u'DEPARTAMENTO', u'DIA',  u'ESCOLARIDAD',
       u'ESTADO CIVIL', u'MOVIL AGRESOR', u'MOVIL VICTIMA',
       u'MUNICIPIO', u'PAIS NACE', u'PROFESIONES', u'SEXO', u'ZONA']
for colname in cols_to_scan:
    data_all[colname] = data_all[colname].apply(myunidecode)

## read offical in municipality and department data to allow cleaning against official designations

In [11]:
data_municipalities = pd.read_csv('Data/municipalities/Departamentos_y_municipios_de_Colombia.csv',encoding='utf8')

In [12]:
cols_to_scan =  [u'REGION', u'DEPARTAMENTO', u'MUNICIPIO']
for colname in cols_to_scan:
    data_municipalities[colname] = data_municipalities[colname].apply(myunidecode)
departamentos = [x.upper() for x in data_municipalities.DEPARTAMENTO.unique()]
dep_to_id = {k:i for i,k in enumerate(sorted(departamentos))} 
id_to_dep = {i:k for k,i in dep_to_id.items()}
municipios = [x.upper() for x in data_municipalities.MUNICIPIO.unique()]
muni_to_id = {k:i for i,k in enumerate(sorted(municipios))} 
id_to_muni = {i:k for k,i in muni_to_id.items()}

## Clean MUNICIPIO and DEPARTAMENTO columns in social leaders data

In [13]:
# remove excessive white spaces
data_social['MUNICIPIO'] = data_social['MUNICIPIO'].apply(remove_excessive_ws)
data_social['DEPARTAMENTO'] = data_social['DEPARTAMENTO'].apply(remove_excessive_ws)

In [14]:
# Correcting individual mistakes
idx = data_social[data_social.DEPARTAMENTO=='POR ESTA'].index[0]
data_social.loc[idx,'DEPARTAMENTO']='GUAVIARE'
idx = data_social[data_social.DEPARTAMENTO=='ARMA DE FUEGO'].index[0]
data_social.loc[idx,'DEPARTAMENTO']='NARINO'

In [15]:
#manual mapping for mistakes that cannot be rectified automatically
to_replace_manual = {'TUMACO':'SAN ANDRES DE TUMACO',
               'SAN JOSE DE APARTADO':'APARTADO',
             'ISCUANDE':'SANTA BARBARA',
            'PUERTO LEGUIZAMO':'LEGUIZAMO',
             'LA HORMIGA':'VALLE DE GUAMEZ',
             'SAN PEDRO DE LOS MILAGROS':'SAN PEDRO',
             'EL BORDO':'PATIA',
             'BARRIO COMUNEROS':'BOGOTA D.C.',
             'TAMBO':'EL TAMBO',
             'BUGA':'GUADALAJARA DE BUGA',
             'COPEY':'EL COPEY',
             'SAN VICENTE DE FERRER':'SAN VICENTE',
             'LA GABARRA':'TIBU',
             'ANTIOQUIA':'MEDELLIN'}
for name,match in to_replace_manual.items():
    data_social['MUNICIPIO'] 
    data_social.loc[data_social['MUNICIPIO']==name,'MUNICIPIO']=match

In [16]:
#use difflib to automatically correct the rest of the municipios. 
# print all that is being replaced. 
import difflib
to_replace_auto = {}
for name in data_social['MUNICIPIO'].unique():
    if not name in muni_to_id:
        match = difflib.get_close_matches(name, municipios, n=1, cutoff=0.5)[0]
        print( name,'->',match)
        to_replace_auto[name]=match
    
for name,match in to_replace_auto.items():
    data_social['MUNICIPIO'] 
    data_social.loc[data_social['MUNICIPIO']==name,'MUNICIPIO']=match

TIRRALTA -> TIERRALTA
BOGOTA -> BOGOTA D.C.
MIRTANDA -> MIRANDA
ALMAGER -> ALMAGUER
TRUJULLO -> TRUJILLO
JARIN -> JARDIN
TAQUI -> TARQUI
BARRANDUILLA -> BARRANQUILLA
YOPAS -> YOPAL
PIZARRO -> PIJAO
CAPOALEGRE -> CAMPOALEGRE
CASTILLO -> EL CASTILLO
BARBACOS -> BARBACOAS
VILLAGRAZON -> VILLAGARZON
MITRASTO -> MISTRATO
AGUA AZUL -> AGUAZUL
CURMARAL -> CUMARAL
DONCELLO -> EL DONCELLO
CALAOTO -> CALOTO
YACANQUIERE -> YACUANQUER
CARMEN DE VIBORAL -> EL CARMEN DE VIBORAL
MAGANGUE. -> MAGANGUE
SAN CIVENTE DEL CAGUAN -> SAN VICENTE DEL CAGUAN
MONTANITAS -> LA MONTANITA
RUOSUCIO -> RIOSUCIO
DOS QUEBRADAS -> DOSQUEBRADAS
SAN JOSE DE GUAVIARE -> SAN JOSE DEL GUAVIARE
CARTAGENA DEL CHIARA -> CARTAGENA DEL CHAIRA
PIVIJAI -> PIVIJAY


In [17]:
#use difflib to automatically correct the rest of the DEPARTAMENTOs. 
# print all that is being replaced. 
to_replace_auto = {}
for name in data_social['DEPARTAMENTO'].unique():
    if not name in dep_to_id:
        match = difflib.get_close_matches(name, departamentos, n=1, cutoff=0.5)[0]
        print (name,'->',match)
        to_replace_auto[name]=match

for name,match in to_replace_auto.items():
    data_social['DEPARTAMENTO'] 
    data_social.loc[data_social['DEPARTAMENTO']==name,'DEPARTAMENTO']=match

VALLE DEL CUACA -> VALLE DEL CAUCA
ANTIOAUIA -> ANTIOQUIA
RISALDA -> RISARALDA
CALLE DEL CAUCA -> VALLE DEL CAUCA
NORTE DE SANTNADER -> NORTE DE SANTANDER
RISARQALDA -> RISARALDA
CUNDIMARCA -> CUNDINAMARCA
BOGOTA -> BOGOTA D.C.
VALLE DEL CAUDA -> VALLE DEL CAUCA
RISARLADA -> RISARALDA
ANTIOQ -> ANTIOQUIA


# Rename columns

In [18]:
data_social.columns = map(myunidecode, data_social.columns)
data_social.rename(columns={u'FECHA DE HECHOS          D/M/A.': u'Date',
                            u'ARMA UTILIZADA': u'ARMA EMPLEADA'}, inplace=True)

# Cleaning additional variables in Social Leaders data

In [19]:
colname = u'CALIDAD DE LA VICTIMA'
data_social[colname] = data_social[colname].apply(myunidecode)
data_social[colname] = data_social[colname].apply(remove_excessive_ws)

colname = u'NOMBRE DE LA ORGANIZACION'
data_social[colname] = data_social[colname].apply(myunidecode)
data_social[colname] = data_social[colname].apply(remove_excessive_ws)

colname = u'TIPO DE ORGANIZACION'
data_social[colname] = data_social[colname].apply(myunidecode)
data_social[colname] = data_social[colname].apply(remove_excessive_ws)

colname = u'ARMA EMPLEADA'
data_social[colname] = data_social[colname].apply(myunidecode)
data_social[colname] = data_social[colname].apply(remove_excessive_ws)
data_social[colname] = data_social[colname].apply(remove_short)

### clean TIPO DE ORGANIZACION

In [20]:
to_replace_manual = {'LGTB':'LGTBI',
               'COMUNITARIA':'COMUNAL',
             'RESTITUCION DE TIERRAS':'RESTITUCION TIERRAS',
            'RECLAMACION TIERRAS':'RESTITUCION TIERRAS',
             'LGBT':'LGTBI',
             'CAMPESINA':'CAMPESINO',
                'COMUNITARIO ':'COMUNAL',
                 'COMUNITARIO':'COMUNAL'}

for name,match in to_replace_manual.items():
    data_social[u'TIPO DE ORGANIZACION'] 
    data_social.loc[data_social[u'TIPO DE ORGANIZACION']==name,u'TIPO DE ORGANIZACION']=match

### Clean ARMA EMPLEADA

In [21]:
to_replace_manual = {'ARMA EFUEGO':'ARMA FUEGO',
               'ARAMA FUEGO':'ARMA FUEGO',
             'ARNA FUEGO':'ARMA FUEGO',
            'ARTMA FUEGO':'ARMA FUEGO',
             'RMA FUEGO':'ARMA FUEGO',
             'AGENAMA FUIGO':'ARMA FUEGO',
                    'ARESAMA FUEGO':'ARMA FUEGO',
                    'ARMA EFUEGO':'ARMA FUEGO',
                    'ARNA FUEGR':'ARMA FUEGO',
                    'ARMAS FUEGO':'ARMA FUEGO',
                    'ARMA FUEGO.':'ARMA FUEGO',
                    'ARMA CORTOPUNZANTE':'ARMA BLANCA',
                    'ELEMENTO CONTUNDENTE':'OBJETO CONTUNDENTE',
                    'POR ESTABLECER':'POR DETERMINAR',
                    'CUASA MUERTE FUE POR AHORACMIENTO':'CUERDA/SOGA/CADENA',
                    'OBJETO CONTUNDENTE VEHICULO':'VEHICULO'
                    }
for name,match in to_replace_manual.items():
    data_social[u'ARMA EMPLEADA'] 
    data_social.loc[data_social[u'ARMA EMPLEADA']==name,u'ARMA EMPLEADA']=match

## Replace and clean national homicides data 

In [22]:
municipios.append('SAN LUIS DE PALENQUE')

In [23]:
to_replace_manual = {'VALLE':'VALLE DEL CAUCA',
                     'GUAINAA':'GUAINIA',
                     'NARIAO':'NARINO',
                     'BOLAVAR':'BOLIVAR',
                     'CARDOBA':'CORDOBA',
                     'GUAJIRA':'LA GUAJIRA',
                     'QUINDAO':'QUINDIO',
                     'CHOCA':'CHOCO',
                     'VAUPAS':'VAUPES',
                     'GUAINAA':'GUAINIA',
             'SAN ANDRES':'ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA',
             'SAN ANDRAS':'ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA'
             }
for name,match in to_replace_manual.items():
    data_all['DEPARTAMENTO'] 
    data_all.loc[data_all['DEPARTAMENTO']==name,'DEPARTAMENTO']=match
    
to_replace_manual = {'BELAN':'BELEN',
                 'MANA':'MANI',
                 'TIBA':'TIBU',
                 'PUERTO LEGUIZAMO':'LEGUIZAMO',
                 'APAA':'APIA',
                 'ATICA':'UTICA',
                 'ACHA':'ACHI',
                 'MANATA':'MANATI',
                 'LLORA':'LLORO',
                 'MITA (CT)':'MITU',
                 'CHAA':'CHIA',
                 'SOPA':'SOPO'}
for name,match in to_replace_manual.items():
    data_all['MUNICIPIO'] 
    data_all.loc[data_all['MUNICIPIO']==name,'MUNICIPIO']=match


In [24]:
#clear white space in name of attribute
data_all.rename(columns={u'DEPARTAMENTO ': u'DEPARTAMENTO'}, inplace=True)

#clear whitespaces and check the proposed mappings
data_all[u'DEPARTAMENTO'] = data_all[u'DEPARTAMENTO'].apply(remove_excessive_ws)
data_all[u'MUNICIPIO'] = data_all[u'MUNICIPIO'].apply(remove_excessive_ws)
to_replace = {}
for name in data_all[u'DEPARTAMENTO'].unique():
    if not name in dep_to_id:
        match = difflib.get_close_matches(name, departamentos, n=1, cutoff=0.5)[0]
        print(name,'->',match) 
        to_replace[name]=match
        
print('-'*20) 
for name in data_all['MUNICIPIO'].unique():
    if not name in muni_to_id:
        result = difflib.get_close_matches(name, municipios, n=1, cutoff=0.5)
        if result:
            match = result[0]
            print(name,'->',match) 
            to_replace[name]=match
        else:
            print(name,'!!!!!!!!! NO MATCH' ) 

--------------------
CIUDAD BOLAVAR -> CIUDAD BOLIVAR
MEDELLAN (CT) -> MEDELLIN
BARRANQUILLA (CT) -> BARRANQUILLA
CARTAGENA (CT) -> CARTAGENA
MANIZALES (CT) -> MANIZALES
FLORENCIA (CT) -> FLORENCIA
SAN JOSA DEL FRAGUA -> SAN JOSE DEL FRAGUA
POPAYAN (CT) -> POPAYAN
TIMBIQUA -> TIMBIQUI
QUIBDA (CT) -> QUIBDO
MONTERAA (CT) -> MONTERIA
BOGOTA D.C. (CT) -> BOGOTA D.C.
RIOHACHA (CT) -> RIOHACHA
NEIVA (CT) -> NEIVA
LEJANAAS -> LEJANIAS
VILLAVICENCIO (CT) -> VILLAVICENCIO
PASTO (CT) -> PASTO
CACUTA (CT) -> CUCUTA
PUERTO ASAS -> PUERTO ASIS
BELAN DE UMBRAA -> BELEN DE UMBRIA
BUCARAMANGA (CT) -> BUCARAMANGA
IBAGUA (CT) -> IBAGUE
CALI (CT) -> CALI
JAMUNDA -> JAMUNDI
CONVENCIAN -> CONVENCION
ARMENIA (CT) -> ARMENIA
QUINCHAA -> QUINCHIA
SAN ZENAN -> SAN ZENON
SANTA MARTA (CT) -> SANTA MARTA
PEREIRA (CT) -> PEREIRA
BOLAVAR -> BOLIVAR
VALLEDUPAR (CT) -> VALLEDUPAR
SAHAGAN -> SAHAGUN
FUNDACIAN -> FUNDACION
PIENDAMA -> PIENDAMO
NAVITA -> NOVITA
SAN JOSA DEL GUAVIARE (CT) -> SAN JOSE DEL GUAVIARE
MOCOA 

# Cleaning arma empleada in homicide data

In [25]:
colname = u'ARMA EMPLEADA'
data_all[colname] = data_all[colname].apply(myunidecode)
data_all[colname] = data_all[colname].apply(remove_excessive_ws)
data_all[colname] = data_all[colname].apply(remove_short)

In [26]:
to_replace_manual = {'ARMA EFUEGO':'ARMA FUEGO',
               'ARAMA FUEGO':'ARMA FUEGO',
             'ARNA FUEGO':'ARMA FUEGO',
            'ARTMA FUEGO':'ARMA FUEGO',
             'RMA FUEGO':'ARMA FUEGO',
             'AGENAMA FUIGO':'ARMA FUEGO',
                    'ARESAMA FUEGO':'ARMA FUEGO',
                    'ARMA EFUEGO':'ARMA FUEGO',
                    'ARNA FUEGR':'ARMA FUEGO',
                    'ARMAS FUEGO':'ARMA FUEGO',
                    'ARMA FUEGO.':'ARMA FUEGO',
                    'ARMA CORTOPUNZANTE':'ARMA BLANCA',
                    'ELEMENTO CONTUNDENTE':'OBJETO CONTUNDENTE',
                    'POR ESTABLECER':'POR DETERMINAR',
                    'CUASA MUERTE FUE POR AHORACMIENTO':'AHORACMIENTO',
                    'OBJETO CONTUNDENTE VEHICULO':'VEHICULO',
                     'ARMA BLANCA CORTOPUNZANTE':'ARMA BLANCA',
                     'CORTANTES':'ARMA BLANCA'  ,
                     'PUNZANTES':'ARMA BLANCA'  ,
                     'ARTEFACTO EXPLOSIVO/CARGA DINAMITA':'EXPLOSIVO',
                     'MINA ANTIPERSONA': 'EXPLOSIVO',
                     'GRANADA MANO':'EXPLOSIVO',
                     'CILINDRO BOMBA':'EXPLOSIVO',
                     'CASA BOMBA':'EXPLOSIVO',
                     'MOTO BOMBA':'EXPLOSIVO',
                     'GRANADA MORTERO':'EXPLOSIVO',
                     'PAQUETE BOMBA':'EXPLOSIVO',
                     'CARRO BOMBA':'EXPLOSIVO'
                    }
for name,match in to_replace_manual.items():
    data_all[u'ARMA EMPLEADA'] 
    data_all.loc[data_all[u'ARMA EMPLEADA']==name,u'ARMA EMPLEADA']=match
    
data_all[u'ARMA EMPLEADA'].value_counts()

ARMA FUEGO                      59643
ARMA BLANCA                     16663
CONTUNDENTES                     2601
EXPLOSIVO                         621
CUERDA/SOGA/CADENA                316
BOLSA PLASTICA                    129
REPORTADO                          87
GASOLINA                           44
SIN EMPLEO ARMAS                   36
CINTAS/CINTURON                    27
VENENO                             27
ALMOHADA                           27
SUSTANCIAS TOXICAS                 15
COMBUSTIBLE                        15
ARTEFACTO INCENDIARIO               9
CINTAS                              8
ACIDO                               4
PRENDAS VESTIR                      4
QUIMICOS                            3
ESCOPOLAMINA                        2
GASES                               1
LIQUIDOS                            1
JERINGA                             1
ROCKET                              1
MEDICAMENTOS                        1
POLVORA(FUEGOS PIROTECNICOS)        1
Name: ARMA E

# Modify dates to proper format in both datasets (very slow)

In [27]:
#function to convert dates to proper format
def convert_dates(dataset,datecol_idx):
    for i in range(0,dataset.shape[0]):
        try:
            dataset.iloc[i,datecol_idx] = pd.Timestamp(parse(dataset.iloc[i,datecol_idx]))
        except IllegalMonthError:
            print("Month") 
            print(i, dataset.iloc[i,datecol_idx]) 
        except IllegalWeekdayError:
            print("Weekday") 
            print(i, dataset.iloc[i,datecol_idx] ) 
        except ValueError:
            print("Value") 
            print(i, dataset.iloc[i,datecol_idx]) 
        except AttributeError:
            print("Attribute") 
            print(i, dataset.iloc[i,datecol_idx]) 
    return dataset

data_social = convert_dates(data_social,11)
data_all = convert_dates(data_all,11)

In [28]:
import os
if not os.path.exists('Data'):
    os.makedirs('Data')

In [29]:
data_social.to_pickle('Data/data_social.pkl')
data_all.to_pickle('Data/data_all.pkl')